In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import Embedding, LSTM, Dense, SimpleRNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [13]:
##load IMDB
max_feature=1000
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=max)

In [3]:
x_train.shape,y_train.shape

((25000,), (25000,))

In [4]:
##Inspect
sample_x=x_train[2]
sample_y=y_train[2]

In [5]:
##Mapping
word_index=imdb.get_word_index()
reverse_word_index=dict([(value,key) for (key,value) in word_index.items()])

for i in sample_x:
    print(reverse_word_index.get(i-3,'?'),end=' ')

1641221/1641221 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
? this has to be one of the worst films of the ? when my friends i were watching this film being the ? audience it was ? at we just ? watched the first half an hour with our ? ? the ? at how bad it really was the rest of the time everyone else in the ? just started talking to each other ? or ? ? into their ? that they actually ? money they had ? working to watch this ? ? for a film it must have looked like a great idea on ? but on film it looks like no one in the film has a ? what is going on crap acting crap ? i can't get across how ? this is to watch save yourself an hour a bit of your life 

In [6]:
sample_y

0

In [27]:
from tensorflow.keras.preprocessing import sequence

max_len=500

x_train=sequence.pad_sequences(x_train,maxlen=max_len)
x_test=sequence.pad_sequences(x_test,maxlen=max_len)
x_train.shape,x_test.shape

((25000, 500), (25000, 500))

In [31]:
y_train = np.asarray(y_train, dtype='int32')
y_train.shape

(25000,)

In [9]:
#TRAIN SIMPLE RNN

In [18]:
max_len = 500
model = Sequential()
model.add(Embedding(max_feature, 64, input_length=max_len))
model.add(SimpleRNN(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Build the model to initialize weights
model.build((None, max_len))  # Input shape: (batch_size, max_len)

# Now, call model.summary()
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_5 (Embedding)              │ (None, 500, 64)             │          64,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ simple_rnn_5 (SimpleRNN)             │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              65 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 72,321 (282.50 KB)

 Trainable params: 72,321 (282.50 KB)

 Non-trainable params: 0 (0.00 B)

In [19]:
#creating an instance of Early stoppinh
from tensorflow.keras.callbacks import EarlyStopping
early_stopping=EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
early_stopping

In [23]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [32]:
## Train the Model
history = model.fit(
    x_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=.2,
    callbacks=[early_stopping]
)

Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 79s 124ms/step - accuracy: 0.5717 - loss: 0.7148 - val_accuracy: 0.6422 - val_loss: 0.6143
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 81s 123ms/step - accuracy: 0.6054 - loss: 426753280.0000 - val_accuracy: 0.5572 - val_loss: 0.7241
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 80s 121ms/step - accuracy: 0.5868 - loss: 0.6663 - val_accuracy: 0.6430 - val_loss: 0.6160
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 83s 123ms/step - accuracy: 0.6634 - loss: 0.5949 - val_accuracy: 0.6720 - val_loss: 0.5948
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 77s 124ms/step - accuracy: 0.7118 - loss: 0.5570 - val_accuracy: 0.7012 - val_loss: 0.5728
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 82s 124ms/step - accuracy: 0.7351 - loss: 0.5291 - val_accuracy: 0.7304 - val_loss: 0.5373
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 81s 123ms/step - accuracy: 0.7752 - loss: 0.4877 - val_accuracy: 0.7788 - val_loss: 0.4714
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 80s 120ms/step - accuracy: 0.8165 -

In [30]:
y_train = np.asarray(y_train, dtype='int32')
y_train.shape

(25000,)

In [33]:
## SAve model File
model.save('simple_rnn_model.h5')

In [34]:
ls

sample_data/  simple_rnn_model.h5
